In [6]:
import re 
import tweepy 
import twitter
from tweepy import OAuthHandler 
from matplotlib import pyplot as plt
import pandas as pd
from TwitterAPI import TwitterAPI
import csv
from collections import Counter
import networkx as nx
import sys
import os
import re
import time
from TwitterAPI import TwitterAPI
import itertools

global max_id

#==============================Twitter Tokens==========================#
consumer_key= 'O062S27yXGUh5y1qtoY3SX6Mc'
consumer_secret= '7GFZ7qjRujuLOiRMWTjeBkTX0iPrC9agRwrMobQUBT15HptyUF'
access_token= '2723937392-wOjCn19YOotqPGjfbp2MGkzGtGIu9oBELpHNa5H'
access_token_secret= 'gpaLAjMXmp4RbCx5AYDVqNwaZMEoPvmEznGyACfUvDbhB'
#======================================================================#


tweets =[]
def get_twitter():
    twitter = TwitterAPI(consumer_key,consumer_secret,access_token,access_token_secret)
    return twitter
twitter = get_twitter()

#request = twitter.request('search/tweets', {'q': 'rahul gandhi','count':200,'lang':'en'})
#tweets = [r for r in request]


def get_tweet(twitter,query):

    list_tweet = []
    #query = "trump"
    new_query = query + " -filter:retweets" #This will help to remove Retweets
    tweets = tweepy.Cursor(twitter.search,q=new_query, tweet_mode='extended', lang="en").items(2000)

    for tweet in tweets:
        tweets_dict = {}
        if tweet.full_text not in tweets_dict:
            tweets_dict['text'] = tweet.full_text
            list_tweet.append(tweets_dict)
    #print(list_tweet)
    #for item in list_tweet:
    #    print(item["text"])
    return list_tweet

#********************************CLEANING OF DATA***************************************
def tweet_cleaner(jargons):
    
    in_list = list(jargons)
    jargons = "".join(in_list)
    jargons = str(jargons.lower())
    text = ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)+"," ",jargons).split())
    #clean_tweet = re.sub(r'@\w+','',text,flags=re.MULTILINE).split()
    clean_tweet = str(text)
    return str(clean_tweet)
#********************************CLEANING OF DATA***************************************

#**********************Create CSV File **************************************
def create_csv_File(query,list_tweet):
    with open("Collected_Data"+os.path.sep+query+".csv",'w') as wp:
        csv_writer = csv.writer(wp)
        csv_writer.writerow(["Tweet Text", "Sentiment Label"])
        for item in list_tweet:
            #print(item)
            clean_tweet = tweet_cleaner(item.values())
            #print(clean_tweet)
            csv_writer.writerow([clean_tweet]) #item.values()

            #csv_writer.writerow([tweet_dict["id_str"],tweet_dict["text"]])
    wp.close()
    collector_details(len(list_tweet))
    print("❂❂❂❂❂❂❂❂❂ File for ",query,"created Successfully! ❂❂❂❂❂❂❂❂❂")
    
def collector_details(No_of_tweets):
   
    with open("Collected_Data"+os.path.sep +"collected_Data.txt",'w') as fp:
        #fp.write("No of Users Collected : " + str(No_of_users) +"\n")
        fp.write("No of messages Collected : " + str(No_of_tweets)+"\n")

    fp.close()    
    

def get_twitter_api():
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_token_secret)
    api = tweepy.API(auth)
    return api

#################################################################################################
#################################################################################################


def authenticate(consumer_key,consumer_secret,access_token,access_token_secret):
    try: 
        # create OAuthHandler object 
        auth = OAuthHandler(consumer_key, consumer_secret) 
        # set access token and secret 
        auth.set_access_token(access_token, access_token_secret) 
        # create tweepy API object to fetch tweets 
        api = tweepy.API(self.auth) 
        twitter = TwitterAPI(consumer_key, consumer_secret, access_token, access_token_secret)
    except: 
        print("E.. Authentication approved!")

    #global twitter
    twitter = TwitterAPI(consumer_key, consumer_secret, access_token, access_token_secret)
    return twitter

def robust_request(twitter, resource, params, max_tries=7000):
    
    """ If a Twitter request fails, sleep for 15 minutes.
    Do this at most max_tries times before quitting.
    Args:
      twitter .... A TwitterAPI object.
      resource ... A resource string to request; e.g., "friends/ids"
      params ..... A parameter dict for the request, e.g., to specify
                   parameters like screen_name or count.
      max_tries .. The maximum number of tries to attempt.
    Returns:
      A TwitterResponse object, or None if failed.
    """
    for i in range(max_tries):
        request = twitter.request(resource, params)
        if request.status_code == 200:   
            return request

def clean_tweet(tweet):
    a =  (re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", str(tweet)).split())
    return " ".join(a)

  
def get_tweets(twitter): 
   
    # empty list to store parsed tweets 
    next_max_id=0
    tweets = [] 
    users = ['realDonaldTrump','HillaryClinton','MichelleObama']
    new_max_ids = [0,0,0]
    outfile = open("Cluster_Folder"+os.path.sep+"cluster_testData"+".csv",'a')
    writer = csv.writer(outfile)
    outfile.write('Text\n')
    outfile.close()
    try: 

        # call twitter api to fetch tweets
        while not all(elem == -1 for elem in new_max_ids):
            for user in users:
                if(new_max_ids[users.index(user)]== -1):
                    continue
                elif (new_max_ids[users.index(user)] == 0):
                    #print('1')
                    fetched_tweets = robust_request(twitter, resource='search/tweets', 
                                                     params={'q': user, 'count' : 100}).json()
                else:
                    resource = 'search/tweets'
                    #print('2')
                    fetched_tweets = robust_request(twitter, resource,
                                         params={'max_id':new_max_ids[users.index(user)],'q': user, 'count' : 100}).json() 

                #print("fetched tweets",fetched_tweets)
                #print("reched 1")
                if 'next_results' in fetched_tweets['search_metadata']:
                    next_results = fetched_tweets['search_metadata']['next_results']
                    next_max_id = next_results.split('max_id=')[1].split('&')[0]
                    new_max_ids[users.index(user)] = next_max_id
                else: 
                    new_max_ids[users.index(user)] = -1
                #print("reached 2")
                unique_list = []
                for k in range(len(fetched_tweets['statuses'])):
                    tw = clean_tweet((fetched_tweets['statuses'][k]['text']))
                    sn = fetched_tweets['statuses'][k]['user']['location']
                    #tweets.append(tw)
                    unique_list.append((tw,sn))
                #print("reached 3")
                outfile = open("Cluster_Folder"+os.path.sep+"cluster_testData"+".csv",'a')
                writer = csv.writer(outfile)
                for tup in unique_list:
                    x = tup[0]
                    y = tup[1]
                    y = y.rstrip('\n')
                    y = y.lstrip('\n')
                    if x.strip().startswith("RT")==False and x!="" and y!="" :        
                        writer.writerow ([x,user,y])
                outfile.close()
                print("A new csv File successfully ..being created inside the folder Cluster_Folder")
                print("Loading...file")
                print("My suggestion would be to wait for a minute atleast then, follow the below")
                print("You have to stop it manually, if you dont want to download the extar data")
               
        return tweets

    except tweepy.TweepError as e: 
        # print error (if any) 
        print("Error : " + str(e)) 
       


    
def main(): 
    
    
    #==============================Twitter Tokens==========================#
    consumer_key= 'O062S27yXGUh5y1qtoY3SX6Mc'
    consumer_secret= '7GFZ7qjRujuLOiRMWTjeBkTX0iPrC9agRwrMobQUBT15HptyUF'
    access_token= '2723937392-wOjCn19YOotqPGjfbp2MGkzGtGIu9oBELpHNa5H'
    access_token_secret= 'gpaLAjMXmp4RbCx5AYDVqNwaZMEoPvmEznGyACfUvDbhB'
    #======================================================================# 
    
    print("*********ENTER  1  FOR DOWNLOADING TEST DATA FOR CLASSIFICATION AND \n ENTER  2  FOR DOWNLOADING CLUSTER DATA FOR CLUSTER CLASSIFICATION*****")
    print("Once you create your file for either classification or clustering , you can rerun the file to download other set of data")
    
    user_input = input()
    
    if user_input == "1":
        
        
        api = get_twitter_api()
        query = ["trump"]
        for i in range(len(query)):
            list_tweet = get_tweet(api,query[i])
            create_csv_File(query[i],list_tweet)
        
          
    ###################################################################################################

    else:
        twitter = authenticate(consumer_key,consumer_secret,access_token,access_token_secret) 

        # calling function to get tweets 
        tweets1 = get_tweets(twitter)

    
if __name__ == "__main__": 
    # calling main function 
    main() 



*********ENTER  1  FOR DOWNLOADING TEST DATA FOR CLASSIFICATION AND 
 ENTER  2  FOR DOWNLOADING CLUSTER DATA FOR CLUSTER CLASSIFICATION*****
Once you create your file for either classification or clustering , you can rerun the file to download other set of data
2
Error: Authentication Failed
fetched tweets {'statuses': [{'created_at': 'Sat Apr 27 03:49:07 +0000 2019', 'id': 1121984596046950400, 'id_str': '1121984596046950400', 'text': 'RT @SpeakerPelosi: .@realDonaldTrump pulling the U.S. out of yet another arms control treaty undermines our national security and makes for…', 'truncated': False, 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [{'screen_name': 'SpeakerPelosi', 'name': 'Nancy Pelosi', 'id': 15764644, 'id_str': '15764644', 'indices': [3, 17]}, {'screen_name': 'realDonaldTrump', 'name': 'Donald J. Trump', 'id': 25073877, 'id_str': '25073877', 'indices': [20, 36]}], 'urls': []}, 'metadata': {'iso_language_code': 'en', 'result_type': 'recent'}, 'source': '<a hre

fetched tweets {'statuses': [{'created_at': 'Sat Apr 27 03:49:06 +0000 2019', 'id': 1121984592393715714, 'id_str': '1121984592393715714', 'text': '@domesude @GH_obsession @bigdinca @EdKrassen @HillaryClinton @PostOpinions No offense.Just seems like the reason pp… https://t.co/DVP1VSwXWe', 'truncated': True, 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [{'screen_name': 'domesude', 'name': 'Some Dude', 'id': 2918883866, 'id_str': '2918883866', 'indices': [0, 9]}, {'screen_name': 'GH_obsession', 'name': '🇺🇸❄️GHObsession ❄️🇺🇸WomansMob', 'id': 3102724793, 'id_str': '3102724793', 'indices': [10, 23]}, {'screen_name': 'bigdinca', 'name': 'Don', 'id': 68905747, 'id_str': '68905747', 'indices': [24, 33]}, {'screen_name': 'EdKrassen', 'name': 'Ed Krassenstein', 'id': 132339474, 'id_str': '132339474', 'indices': [34, 44]}, {'screen_name': 'HillaryClinton', 'name': 'Hillary Clinton', 'id': 1339835893, 'id_str': '1339835893', 'indices': [45, 60]}, {'screen_name': 'PostOpinions', 'na

fetched tweets {'statuses': [{'created_at': 'Sat Apr 27 03:47:22 +0000 2019', 'id': 1121984152335732737, 'id_str': '1121984152335732737', 'text': '"Failure is a feeling long before it becomes an actual result. It’s vulnerability that breeds with self-doubt and t… https://t.co/W6AjUyWtq9', 'truncated': True, 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [], 'urls': [{'url': 'https://t.co/W6AjUyWtq9', 'expanded_url': 'https://twitter.com/i/web/status/1121984152335732737', 'display_url': 'twitter.com/i/web/status/1…', 'indices': [117, 140]}]}, 'metadata': {'iso_language_code': 'en', 'result_type': 'recent'}, 'source': '<a href="https://mobile.twitter.com" rel="nofollow">Twitter Web App</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 35322022, 'id_str': '35322022', 'name': 'Lady Maverick NYC ☔️', 'screen_name': 'LadyMaverickNYC', 'location': '

fetched tweets {'statuses': [{'created_at': 'Sat Apr 27 03:48:53 +0000 2019', 'id': 1121984536873775104, 'id_str': '1121984536873775104', 'text': "RT @DeepStateExpose: @realDonaldTrump Retweet: The Clinton's Global Cocaine Smuggling Operation revealed !!!\n#DrainTheDeepStateSwamp https:…", 'truncated': False, 'entities': {'hashtags': [{'text': 'DrainTheDeepStateSwamp', 'indices': [109, 132]}], 'symbols': [], 'user_mentions': [{'screen_name': 'DeepStateExpose', 'name': 'Deep State Exposed® ⭐⭐⭐⭐⭐', 'id': 3580248555, 'id_str': '3580248555', 'indices': [3, 19]}, {'screen_name': 'realDonaldTrump', 'name': 'Donald J. Trump', 'id': 25073877, 'id_str': '25073877', 'indices': [21, 37]}], 'urls': []}, 'metadata': {'iso_language_code': 'en', 'result_type': 'recent'}, 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply

fetched tweets {'statuses': [{'created_at': 'Sat Apr 27 03:43:02 +0000 2019', 'id': 1121983061959348226, 'id_str': '1121983061959348226', 'text': '@kaloshilo @Sha_Elise24 @SenSanders @HillaryClinton Bernie or Bust was definitely a movement within his supporters,… https://t.co/453vbGX2XY', 'truncated': True, 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [{'screen_name': 'kaloshilo', 'name': 'kaloshilo', 'id': 16164138, 'id_str': '16164138', 'indices': [0, 10]}, {'screen_name': 'Sha_Elise24', 'name': "Queen Bey's Imaginary Backup Dancer", 'id': 338099920, 'id_str': '338099920', 'indices': [11, 23]}, {'screen_name': 'SenSanders', 'name': 'Bernie Sanders', 'id': 29442313, 'id_str': '29442313', 'indices': [24, 35]}, {'screen_name': 'HillaryClinton', 'name': 'Hillary Clinton', 'id': 1339835893, 'id_str': '1339835893', 'indices': [36, 51]}], 'urls': [{'url': 'https://t.co/453vbGX2XY', 'expanded_url': 'https://twitter.com/i/web/status/1121983061959348226', 'display_url': 'twitte

...

fetched tweets {'statuses': [{'created_at': 'Sat Apr 27 03:48:40 +0000 2019', 'id': 1121984481735237632, 'id_str': '1121984481735237632', 'text': 'RT @Lrihendry: @realDonaldTrump @NRA Hillary said the people must stand up to the NRA, \n\nPresident Trump realizes the NRA is we the people…', 'truncated': False, 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [{'screen_name': 'Lrihendry', 'name': 'Lori Hendry', 'id': 129235890, 'id_str': '129235890', 'indices': [3, 13]}, {'screen_name': 'realDonaldTrump', 'name': 'Donald J. Trump', 'id': 25073877, 'id_str': '25073877', 'indices': [15, 31]}, {'screen_name': 'NRA', 'name': 'NRA', 'id': 21829541, 'id_str': '21829541', 'indices': [32, 36]}], 'urls': []}, 'metadata': {'iso_language_code': 'en', 'result_type': 'recent'}, 'source': '<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_

fetched tweets {'statuses': [{'created_at': 'Sat Apr 27 03:36:14 +0000 2019', 'id': 1121981354235826177, 'id_str': '1121981354235826177', 'text': '@Emolclause @JoeBiden @JoeBiden / @HillaryClinton', 'truncated': False, 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [{'screen_name': 'Emolclause', 'name': 'Emoluments Clause', 'id': 802649176299466752, 'id_str': '802649176299466752', 'indices': [0, 11]}, {'screen_name': 'JoeBiden', 'name': 'Joe Biden', 'id': 939091, 'id_str': '939091', 'indices': [12, 21]}, {'screen_name': 'JoeBiden', 'name': 'Joe Biden', 'id': 939091, 'id_str': '939091', 'indices': [22, 31]}, {'screen_name': 'HillaryClinton', 'name': 'Hillary Clinton', 'id': 1339835893, 'id_str': '1339835893', 'indices': [34, 49]}], 'urls': []}, 'metadata': {'iso_language_code': 'und', 'result_type': 'recent'}, 'source': '<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>', 'in_reply_to_status_id': 1121417724687679488, 'in_reply_to_status_i

fetched tweets {'statuses': [{'created_at': 'Sat Apr 27 00:31:11 +0000 2019', 'id': 1121934783682977792, 'id_str': '1121934783682977792', 'text': 'RT @UCLAnewsroom: Michelle Obama and dozens of celebrities will be on campus next week - UC and Reach Higher are hosting @MichelleObama’s C…', 'truncated': False, 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [{'screen_name': 'UCLAnewsroom', 'name': 'UCLA Newsroom', 'id': 14621206, 'id_str': '14621206', 'indices': [3, 16]}, {'screen_name': 'MichelleObama', 'name': 'Michelle Obama', 'id': 409486555, 'id_str': '409486555', 'indices': [121, 135]}], 'urls': []}, 'metadata': {'iso_language_code': 'en', 'result_type': 'recent'}, 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 3044762034, 'id_str': '3044762034', '

fetched tweets {'statuses': [{'created_at': 'Sat Apr 27 03:48:26 +0000 2019', 'id': 1121984420649603072, 'id_str': '1121984420649603072', 'text': "@BluntsvilleTx @dannielachance @BecomesStar @WhiteHouse @realDonaldTrump Hmm didn't bother to read the list,  does… https://t.co/HiJWyXbzVY", 'truncated': True, 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [{'screen_name': 'BluntsvilleTx', 'name': 'BluntsvilleTxRadio', 'id': 519836942, 'id_str': '519836942', 'indices': [0, 14]}, {'screen_name': 'dannielachance', 'name': 'Dannie LaChance', 'id': 2820715856, 'id_str': '2820715856', 'indices': [15, 30]}, {'screen_name': 'BecomesStar', 'name': 'River Becomes The Star', 'id': 1109959725205065728, 'id_str': '1109959725205065728', 'indices': [31, 43]}, {'screen_name': 'WhiteHouse', 'name': 'The White House', 'id': 822215673812119553, 'id_str': '822215673812119553', 'indices': [44, 55]}, {'screen_name': 'realDonaldTrump', 'name': 'Donald J. Trump', 'id': 25073877, 'id_str': '25073877

KeyboardInterrupt: 